In [1]:
import os
from urllib import request 
from collections import defaultdict
import numpy as np
import joblib
import pandas as pd

## Create a folder to save data files.

In [5]:
if not os.path.isdir('data'):
    os.makedirs('data')

In [15]:
def download_data(name_dir="data", filename="data.csv",
                  data_url = 'https://www2.census.gov/programs-surveys/popest/datasets/2010-2017/cities/totals/sub-est2017_42.csv'):
    """ 
    Download both train and test dataset to a directory named "data". 

    If the directory doesn't exist, this function will create one. 

    Parameters
    ----------
    data_url : str, default 'https://www2.census.gov/programs-surveys/popest/datasets/2010-2017/cities/totals/sub-est2017_42.csv'
        The URL of the location of the file.
    filename : str, default 'data.csv'
    name_dir : str, default 'data'
        Name of the directory.
    
    Returns
    -------
    No return. Create a directory if needed and 
    download the file to that directory.
    """
    
    if not os.path.isdir(name_dir):
        os.makedirs(name_dir)

    data_path = os.path.join(name_dir, filename)

    # if file doesn't exist, download it.
    if not os.path.isfile(data_path):
        request.urlretrieve(data_url,data_path)

# https://stackoverflow.com/questions/25010369/wget-curl-large-file-from-google-drive
import requests

def download_file_from_google_drive(id, destination):
    def get_confirm_token(response):
        for key, value in response.cookies.items():
            if key.startswith('download_warning'):
                return value

        return None

    def save_response_content(response, destination):
        CHUNK_SIZE = 32768

        with open(destination, "wb") as f:
            for chunk in response.iter_content(CHUNK_SIZE):
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)

    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def save_obj(obj, name, name_dir='data' ):
    """
    Save to pickle.
    
    Parameters
    ----------
    obj : any object 
        This can be a dictionary or ndarray.
    name : str
        The name for the object to be saved.
    name_dir : str, default 'data'
        Name of the directory.
    
    Returns
    -------
    No return.
        Save the pickle object to the local file system.
    """
    
    if not os.path.isdir(name_dir):
        os.makedirs(name_dir)

    data_path = os.path.join(name_dir, name+'.pkl')
    
    with open(data_path, 'wb') as f:
        joblib.dump(obj, f)

def load_obj(name, gdrive_id, name_dir='data' ):
    """
    Load the pickle object from the local file system.
    
    Parameters
    ----------
    name : str
        The name for the object to be saved.
    gdrive_id : str
        The google drive id for the data.
    name_dir : str, default 'data'
        Name of the directory.
    
    Returns
    -------
    object
        Return an object such as a dictionary.
    """
    data_path = os.path.join(name_dir, name+'.pkl')
    
    # if file doesn't exist, download it.
    if not os.path.isfile(data_path):
        download_file_from_google_drive(gdrive_id, data_path)
        
    with open(data_path, 'rb') as f:
        return joblib.load(f)
    
def nested_dict():
  """
  This returns a empty nestable dictionary.
  """
  return defaultdict(nested_dict)


## Download Sub-county population dataset for PA

In [7]:
# https://www.census.gov/data/datasets/2017/demo/popest/total-cities-and-towns.html
url_pop = 'https://www2.census.gov/programs-surveys/popest/datasets/2010-2017/cities/totals/sub-est2017_42.csv'

In [8]:
download_data(name_dir="data", 
              filename="population.csv",
              data_url=url_pop)

In [12]:
df_pop = pd.read_csv('data/population.csv')
df_pop.head()

,SUMLEV,STATE,COUNTY,PLACE,COUSUB,CONCIT,PRIMGEO_FLAG,FUNCSTAT,NAME,STNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016,POPESTIMATE2017
0,40,42,0,0,0,0,0,A,Pennsylvania,Pennsylvania,12702379,12702857,12711063,12742811,12768034,12778450,12790341,12791124,12787085,12805537
1,162,42,0,116,0,0,0,A,Abbottstown borough,Pennsylvania,1011,1011,1010,1011,1008,1006,1009,1012,1013,1020
2,162,42,0,332,0,0,0,A,Adamsburg borough,Pennsylvania,172,172,172,171,170,169,168,167,165,164
3,162,42,0,364,0,0,0,A,Adamstown borough,Pennsylvania,1789,1789,1801,1835,1837,1839,1844,1845,1847,1850
4,162,42,0,396,0,0,0,A,Addison borough,Pennsylvania,207,207,207,205,204,202,201,198,198,197


In [13]:
df_pop['NAME_UP'] = df_pop['NAME'].str.upper() #  .str.replace('[^\w\s]','')
df_pop.head()


,SUMLEV,STATE,COUNTY,PLACE,COUSUB,CONCIT,PRIMGEO_FLAG,FUNCSTAT,NAME,STNAME,...,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016,POPESTIMATE2017,NAME_UP
0,40,42,0,0,0,0,0,A,Pennsylvania,Pennsylvania,...,12702857,12711063,12742811,12768034,12778450,12790341,12791124,12787085,12805537,PENNSYLVANIA
1,162,42,0,116,0,0,0,A,Abbottstown borough,Pennsylvania,...,1011,1010,1011,1008,1006,1009,1012,1013,1020,ABBOTTSTOWN BOROUGH
2,162,42,0,332,0,0,0,A,Adamsburg borough,Pennsylvania,...,172,172,171,170,169,168,167,165,164,ADAMSBURG BOROUGH
3,162,42,0,364,0,0,0,A,Adamstown borough,Pennsylvania,...,1789,1801,1835,1837,1839,1844,1845,1847,1850,ADAMSTOWN BOROUGH
4,162,42,0,396,0,0,0,A,Addison borough,Pennsylvania,...,207,207,205,204,202,201,198,198,197,ADDISON BOROUGH


## Download Preprocessed EMS Dispatch Data
- `df_ems` can be regenerated by using `data_preprocessing.ipynb`.

In [16]:
# Save data after filling missing values.
# save_obj(df_ems,'df_ems')
# https://drive.google.com/open?id=1OCt0jm--zCE_mTKAub_Ddo4W6BayKQ0B
df_ems = load_obj('df_ems','1OCt0jm--zCE_mTKAub_Ddo4W6BayKQ0B')

In [17]:
df_ems.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 780697 entries, 0 to 780697
Data columns (total 11 columns):
Call_ID_Hash         780697 non-null object
SERVICE              780697 non-null object
PRIORITY             780697 non-null object
PRIORITY_DESC        780697 non-null object
AGENCY               780697 non-null object
CALL_QUARTER         780697 non-null object
CALL_YEAR            780697 non-null int64
DESCRIPTION_SHORT    780697 non-null object
CITY_CODE            780697 non-null object
CITY_NAME            780697 non-null object
GEOID                780697 non-null int64
dtypes: int64(2), object(9)
memory usage: 71.5+ MB


In [18]:
# Rebuild the city name list
ct_names = df_ems.CITY_NAME.unique().tolist()

In [19]:
def get_pop(ct_name, df, name_col, yr_st, yr_end):
    columns=['POPESTIMATE{yr}'.format(yr=str(yr)) 
             for yr in range(yr_st, yr_end+1)]
    ct_ext = ['BOROUGH', 'CITY', 'TOWNSHIP', 'MUNICIPALITY']
    if ct_name.endswith(('BOROUGH', 'CITY', 'TOWNSHIP', 'BORO')):
        alist = ct_name.split()
        ct_name = ' '.join(alist[:-1])
    if 'CLAIR' in ct_name:
        ct_name = ct_name.replace('ST ', 'ST. ')
    ct_names = ['{name} {ext}'.format(name=ct_name, ext=ext) 
                for ext in ct_ext]
    return (df[(df[name_col]
                .isin(ct_names))][columns]
            .median().astype('int').tolist())
    

In [20]:
get_pop('SOUTH FAYETTE',df_pop,'NAME_UP',2015,2017)

[15423, 15511, 15614]

In [21]:
def get_pop_dict(ct_names, df, name_col, yr_st, yr_end):
    pop_dict = nested_dict()
    for ct_name in ct_names:
        try:
            pops = get_pop(ct_name, df, name_col, yr_st, yr_end)
            for yr,pop in zip(range(yr_st,yr_end+1), pops):
                pop_dict[ct_name][yr]=pop
        except ValueError:
            print(ct_name)
            return ct_name
    return pop_dict

In [22]:
pop_dict = get_pop_dict(ct_names,df_pop,'NAME_UP',2015,2017)
# # load pop_dict from the local system.
# pop_dict = load_obj('pop_dict')

In [23]:
# Save pop_dict to a local file system.
save_obj(pop_dict, 'pop_dict')